# Loudness Level Labeling with the Dictionary Approach

Given an automatically annotated corpus of sound events, the annotations are cleaned from frequent false positives by string matching. 
Then, the remaining annotations are extracted and prepared as a csv file with the extracted sound event span, sound classification, and file name.
The csv is then opened as a pandas dataframe, in which a copy of the sound event spans is lemmatized and lowercased in preparation for the matching algorithm, which detects sound words that have a loudness level assigned to them in the loudness level dictionary. 
In a final step, after averaging the matched sound-word loudness value pairs, the average loudness level value of a sound event is written back into the revised XML TEI files from the beginning of the script. 

In [16]:
import os
import csv
import pandas as pd
import regex as re
from pathlib import Path
from collections import Counter
import xml.etree.ElementTree as ET
import spacy
# Load the German medium model
nlp = spacy.load('de_core_news_md')

In [17]:
#Define the path to the input folder and the name and path of the output file

folder_path = '/Users/sguhr/Desktop/Diss_notebooks/Diss_data_notebooks_man_anno/20240501_corpus_loudness_labeling/20240501_Subcorpus_1856-60_predicted_test'
output_file = '/Users/sguhr/Desktop/Diss_notebooks/Diss_data_notebooks_man_anno/20240501_corpus_loudness_labeling/20240509_sound_spans_output_Subcorpus_predicted_test.csv'

## Automated Revision of frequent false positive Annotations

The following cell is very long because it contains the collection of false positives noticed in the revision of sound events annotated with the round 2 model. As a continuing list of strings where I could add a new false positive case every time I noticed it, the string became longer and longer.

In [18]:
# This works! But it is a very long string with sample false positives from the revisions of the round 2 model annotation output! But hey, the more false positive matches, the better! 

strings = ['', 'dieser nicht gesagt', 'die schönen Lippen', '’s Hand', 'Ihr', 'Der Beamte war', ', fort', 'des Gensd’armen Schmidt', 'alte Mistwagen', 'ihm stehen bleibend', 'Sie wartete keine Antwort ab', 'sein zimmer', 'ein richtig', 'nur etwas zu klein menschenzimmer', 'man über der gleich', 'wie', 'der kompliziert schloß aufsperren', 'der er damit verursachen haben', 'aber', 'wie ein groß', 'von aller seite', 'an der scheibe', 'mutter-', 'in der sich aber', 'wie von unten her', 'ein nicht zu unterdrückend', 'beschränken sich aber bei dieser umstand darauf', 'aber mit der faust', 'bemühen sich', 'befangen in irgendeiner unsinnig hoffnung', 'sie', 'und', 'und haben ihnen', 'und haben nichts', 'immer wieder', 'wenn jetzt der mutter', 'auf gregor zimmer zeigend', 'und', 'trotz der beschwörend erhoben hand der mutter', 'der schwester', 'mit', 'und', 'der vater scheinen es für nötiger zu halten --', 'statt gregor zu vertreiben', 'inzwischen haben der schwester der verlorenheit', 'überwinden', 'der sich der zimmerherr unter der drängen der vater schon schnell näherten', 'der stark nicken sein wie haltlos kopf', 'und ein', 'sich geben', 'der mutter', 'und in ein für gregor gänzlich unverständlich schrecken verlassen der schwester sogar der mutter', 'und ein', 'grete', 'herr samsa nicken ihm bloß mehrmals kurz mit groß auge zu', 'ziehen ihr stöcke aus der stockbehälter', 'verbeugten sich', 'während der schreiben kommen der bedienerin herein --', 'wehren dieser mit ausgestreckt hand entscheiden ab', 'erinnern sie sich an der groß', 'sie haben', 'bequem auf ihr sitz zurücklehnen', 'der aussicht für der zukunft', 'sie', 'dampfend wie ein schlötchen', 'und', 'und haben ihnen', 'ohne anzuklopfen', 'und haben nichts', 'immer wieder', 'Ohne anzuklopfen', 'und er hörte auch', ', wie zuerst,', 'und', 'aber', 'ab', 'aber die', 'aber er', 'aber sie', 'Alle', 'alle', 'als', 'Als', 'Und', 'Aber', 'am', 'Am', 'an', 'An', 'alten', 'Adelens', 'Adele', 'Agnes,', 'als Alice', 'als dieser', 'als er', 'als sie', 'als diese', 'als ich', 'als man', 'Als man', 'Als sie', 'Als er', 'Ameile', ',', '.', ';', ':', '«', '»', '–', '-', 'Dagobert zuckte in ganz perfider Art die Achsel', 'und erhob mich sodann', 'und nun beliebte er', 'und erwiderte später den fragenden Blick des Wirtes mit Achselzucken', 'Der Priester machte eine krampfhafte Bewegung', 'mein Herzschlag stockt', 'Er entliess sie mit einer leichten Geste', 'aus dessen Mienen die Freude lachte', 'Deiner Frau ein kurzes Wort von Deiner Abhaltung zu telegraphiren', 'und grüßte mich mit beiden Armen', 'Als aber jetzt die beiden Gatten einen Willkommenskuß tauschten', 'Auf einmal gingen mir ganz in dem Tonfall, wie ich sie gehört, die Worte durch den Sinn', 'Er nickte', 'achselzuckend ab', 'Alle zehn Schritt grüßte ihn ein Begegnender', 'Er', 'aber ich nickte ihm zu', 'über die Geistlichkeit der frommen Stadt', 'Mit einem Ruck stieß ich den Biedermann von', 'hatte gerade', 'Jetzt ließ sie mich mit einem Knicks', 'Erst unterwegs ward sie ein wenig ruhiger', 'Du bedarfst der Ruhe', 'jähem Ruck', 'ich grüßte empor', 'Ein leichtes resedenes Kleid umschmiegte sie wie zartes Gewölk', 'Ich nickte schämig lächelnd', 'und lächelte höhnisch', 'und zuckte mit den Achseln', 'einem Ruck', 'Ich hole die Laute', 'Sie nickte', 'sie nahm die Laute', 'Lippen der schönen Frau', 'aber er sagt es ihr morgen', 'Seitdem neckte Doctor Matthias seine Schwester nicht mehr auf verletzende Weise', 'Ein Theil desselben meinte', 'sprach englisch und französisch', 'wie deutsch', 'Natürlich sprach sie darüber nicht', 'so fragte diese auch nicht darnach', 'Der Anfall von Schwäche war soeben überwunden', 'Er stand still', 'Er stand still', 'Zerstreut nickte er', 'stutzte der Doctor plötzlich beim Anblicke', 'Er richtete sie ganz ruhig in die Höhe', 'Der Doctor nickte bestätigend mit dem Kopfe', 'Der Doctor belächelte die kindische Sicherheit', 'Dazu kamen die Vorwürfe seines Gewissens in Bezug auf Frau von Dahlhorst', 'Du stockst', 'Der junge Mann zuckte bedauernd die Achseln', 'Sie stockte', 'Dabei legte sich jedoch bleischwer die Frage auf sein Herz', 'dann aber meinte sie zu bemerken', 'Endlich stand er still', 'der sich deutlich in ihrer ganzen Haltung und in dem groß und fest aufgeschlagenen Auge aussprach', 'Beherrscht von ihrer tiefen, streng versteckten Zärtlichkeit stand sie an der Schwelle still', 'Er stand abermals still', 'und zögerte', 'um irgend ein böses oder gutes Wort zu sprechen', 'Da stutzte Elisabeth', 'Er schloß ihr den Mund mit einem Kusse', 'Ein stiller Musikant', 'Er nickte eifrig', 'und er nickte freundlich', 'Mann', 'Er nickte', 'sogar', '»', 'Gedanken ihre Stimme mit', 'Ich drohte ihr zwar mit dem Finger', 'und', 'als nicke mir der große Meister zu', 'Sie hatten', 'Aber ich beruhigte mich', 'Er nickte', '«', 'Aber', 'aber Freund Valentin hatte sich diesmal eine kleine Änderung gestattet', 'denn der alte Asmus sprach in jenem Gedichte doch nur von seiner eigenen Genesung', 'Augen', 'Er nickte eifrig', 'Noch einmal durch einen gemeinsamen Bekannten erhielt ich einen Gruß von Valentin', 'hinter denen allmählich die des stillen Musikanten ganz verschwunden war', 'der alte Herr', 'meiner Seite nickte immer nachdrücklicher mit dem Kopfe', 'Hübsche Stimme', 'Und mit einer höflichen Bewegung sich zu mir wendend', 'Zwar machte sie damals von sich reden', 'Sie nickte lächelnd', 'und', 'Sagst', 'und kroch schläfrig', 'Aber', 'atmen', 'verluderten', 'Der Schmied nickte', 'Er stand ganz still', 'Schibes wedelte zustimmend und mit Bedauern', 'Der Schmied stand still', 'nickte sie', 'staunte er', 'nickte sie', 'Er sagte es nicht', 'Mit einem jähen Ruck', 'als er grüßte', 'Er zitterte schweißbedeckt', 'Sie bat nicht', 'Sie weinte nicht', 'wedelte wie toll', 'fort –', 'Glück', 'und quälte', 'Jan grüßte schon von weitem mit der Hand, so selbstverständlich', 'tänzelnde Stute', '(Jan sagte nie ›aufs Schloß‹', 'lächerigte', 'bitte', 'und beißt', 'und', 'wedelte er', 'jähen Sätzen durch die Hütte hetzend', 'Der Alte schob ruhig die Pelzmütze', 'habe er', 'nickte er', 'Sie zuckte die Achseln', 'Nun streckte', 'Dame', 'Leberfleckchen', 'Im Vorübergehen hatte nur der Funker vor dem Deckoffizier salutiert', 'Die junge Dame nickte ergriffen', 'und', 'zögerte noch immer', 'trat er seinem Bruder entgegen', 'wenn', 'und liegt dort ganz still', "lieg' aber auch rein still", 'nickte Onkel Pökel', 'Das hörte Frau Sellentin', 'Petiskussen überein', 'und', 'aber dies bestärkte Krischan immer mehr in seiner Meinung', 'tat einen kleinen Schluck', 'Helene lauschte gespannt', 'Namen', 'musikalische Helene angenehm durchbebte', 'bang die möglichen Fragen des Herrn Professors fürchtend', 'wenn ich', 'denn sie begann den Sinn seiner Rede zu ahnen', 'Sie mußte antworten', 'und', 'Der Professor verbeugte sich artig', 'Frau', 'athmete Helene innerlich auf', 'diese letzte Frage ließ sich wenigstens im allgemeinen beantworten', 'väterliches Haus', 'und stirnrunzelnd', 'ich telegraphirte', 'So telegraphire einfach an diesen Professor zurück', 'telegraphirte ich nach Leipzig an ihren Mann', 'Und eine innere Stimme antwortete', 'Sich frostig verbeugend', 'Ohne dem Professor zu antworten', 'und', 'um sie zu beruhigen', 'Ohne auf diese Bemerkung einzugehen,', 'daß er ihr schmunzelnd', 'drohte Emma lächelnd mit aufgehobenem Finger', 'begann sie dann mit zwei zierlichen Verbeugungen', 'still', 'Sie', 'Ein', 'auf Emma', 'Und dabei lehnte der arme Sänger', 'Wirklichkeit sein Haupt leise auf die hohe Schulter seines Freundes', 'und lauschte', 'Die geschminkte und gepuderte Excellenz', 'nie entglitt ihr ein lautes Wort, ein Scherz, ein Lachen', 'Fort mit dem steifen » Sie «', 'besonders', 'und', 'und dann', 'sich die', 'unerklärlichen', 'so', 'aber Niemand hörte mich', 'das Orchester müde', 'und setzten ihren Weg fort,', 'zu bewegt,', 'um ein lautes Wort zu sprechen', 'die', 'wiegten', 'Ich habe gerungen', 'Der bejammernswerthe Gatte und Vater war der Verzweiflung nahe', 'Kind auf einem stillen', 'man', 'Die Sterbende hörte sie', 'man hatte keinen Laut gehört', 'Ein Kampf schien in ihr zu toben', 'Die Kinder hatten Glöckchen, kleine Thiere, Häuser, Stuben', 'Später hörten wir nichts weiter von ihr', 'da ich so Dich', 'da pfupften sie sehr', 'Das', 'es schöpfte',  'es sogar einer Magd', 'Als', 'es brenne', 'noch', 'hieß', 'dem Tumulte', 'das zweite Mal', 'Morgen nach deren Abreise', 'Er', 'Der Unheimliche sah ihn ruhig an', 'man unwillkürlich', 'ein Entschluß reifte in ihm, ruhig und kalt', 'Der Baron Benzing saß wieder ruhig', 'wie um', 'war sie wieder', 'daß', 'Es', 'Ich hielt in meinem Gange inne', 'Therese theilt meine Befürchtung', 'und man empfinde das Dröhnen und Ächzen herein', 'Dieses » Nein «', 'Der listig aussehende Knabe', 'Athem', 'Die', 'war aber ohne alle Einrede eine', 'und setzte, ohne mir zu danken,', 'Und eiserne', 'donnernder Tadel', 'Worte', 'rief', 'Fluch,', 'Hilfe an', 'zwischen', 'Da', 'Haus', 'sie', 'Und', 'Die', 'Der', 'Das', 'Ein', 'Eine', 'Diese', 'Dieser', 'Heute hat er', 'die beiden Alten', 'doch so oft', 'In dem Hause des Apothekers ward nur', 'was den Gast', 'des Todes in den Lauten ihres Landes, ihrer Eltern, ihrer Kindheit, ihres Glückes', 'ihm', 'ihr', 'In', 'da', 'die Mädchen grüßten verschämt wieder', 'und wiegte den Kopf hin und her', 'Ungeduldig schüttelte sie seine Hand ab', 'dann warf sie die Lippen mit einem verächtlichen Zucken auf', 'und die blumengeschmückten, blonden und braunen Köpfe zu schüchternem Gruße neigend', 'ein ernstes Händeschütteln', 'die ihn begrüßten', 'Man zuckte mit den Schultern', 'Man grüßte sich diskret', 'ihr die Hand zu reichen', 'die niedliche Kleine zu sich winkte', 'Mit einem unerträglichen Zittern in den Knieen begab das Mädchen sich an ihren Platz zurück', 'die Finger zitterten ihr', 'sie winkte', 'wie das Mädchen am ganzen Leibe bebte', 'Eugenie küßte Agathe leidenschaftlich', 'wurde von ihm gegrüßt', 'winkte Mama sie plötzlich heran', 'Aber das kleine Hausmädchen schüttelte den Kopf', 'während seine dürre lange Hand sie herzlich begrüßte', 'Die Bewegung, mit der er grüßte', 'Ein heftiges, anhaltendes Zittern lief durch ihren Körper', 'und blickte mit geöffneten, bebenden Lippen empor', 'um sie zu begrüßen', 'indem sie ihre Mutter küßte', 'und grüßte nicht', 'seine breiten Schultern zuckten', 'und er mit eiligem Gruß an ihr vorüber wollte', 'gab ihm die Hand', 'Er schüttelte ihr sehr herzlich die Hand', 'Mit zitternden Knieen ging sie nach der Thür', 'Die Hand wurde ihr geschüttelt', 'und nickte mit dem Kopfe', 'Er schüttelte den Kopf', 'und küßte ihn auf die Stirn', 'Küßte ihn auf die Stirn', 'Sie reichte auch der Daniel die Hand – ganz mechanisch', 'ihr Mund begann zu zittern', 'Agathe neigte den Kopf', 'Er küßte sie auf die Stirn', 'und dem Kollegen die Hand schüttelte', 'Agathe erhob sich zitternd', 'Sie zitterte stärker', 'Der alte Sanitätsrat erhielt ein Kopfnicken', 'Raikendorf reichte ihr die Hand mit einem zärtlich zögernden Druck', 'Er drückte ihr die Hand', 'Er küßte Agathe beim Abschied die Hand', 'Er preßte ihre Hand', 'winkte er seiner Frau', 'Agathe überfiel ein Zittern', 'und küßte das Mädchen auf die Stirn', 'Wiesing schüttelte ganz wenig den Kopf', 'und küßte ihn stürmisch', 'Die Schwägerinnen küßten einander', 'in dem der Regierungsrat Heidling seinen Neffen begrüßte', 'Agathe schüttelte den Kopf', 'Zitternd blieb sie stehen', 'Der Regierungsrat küßte Eugenien in warmer Dankbarkeit die Hand', 'Wie eine Sensitive erzitterte sie unter seinen scharfen Augen', 'Und triumphierend hatte sie rings umher gegrüßt und gewinkt', 'Ausbündige Eleganz grüßte', 'zwinkerte Flametti', 'schluckte den Speiserest', 'und Mary hatte gegähnt', 'und erwiderte sehr belustigt die Zeichen des mit dem Kopf andeutenden Schlangenmenschen', 'und schüttelte abweisend die erhobene Hand', 'Und nestelte zitternd an ihrer Bluse', 'Beide nickten, Frau Häsli so hastig', 'winkte Frau Häsli ab', 'und reichte Flametti die Hand', 'Lena schüttelte den Kopf', 'Er deutete mit dem Kopf nach den beiden sacht gehenden Beamten', 'nickte mit dem Kopfe in einer weltmännisch-gewitzigten Weise', 'und wiegte den Kopf', 'der', 'schüttelte den Kopf', 'Aber dann schüttelte er ablehnend den Kopf', 'schüttelte Mechmed versunken den Kopf', 'winkte mit dem Kopfe', 'und streckte Herrn Schnabel die Hand zu über den Tisch', 'lud Flametti mit einer raschen, geschickten Handbewegung ein', 'und Flametti reichte Herrn Rotter indianisch die Hand', 'Raffaëla schüttelte den Kopf ob solcher Unglaublichkeiten', 'Sie lächelte kopfschüttelnd', 'den Herrn Farolyi vom Zirkus Donna Maria Josefa mit vorgestreckter Hand fachmännisch begrüßte', 'und stellte mit einer seitlichen Handbewegung den Pianisten vor', 'Jenny winkte Mutter Dudlinger zu', 'Herr Pips seinerseits versuchte mit plötzlichen, wohlorientierten und freudige Überraschung bekundenden Gesten Jennymama zu bedeuten', 'und winkte ab mit der flachen Hand', 'Traute zuckte die Achseln, mitleidig', 'und schnickte mit dem Kopfe', 'zeigte Jenny auf das verhandelnde Paar', 'Donna Maria Josefa winkte mit Flatterhand', 'dann abwinkend', 'Frau Schnepfe schüttelte den Kopf ob solchen Tumults', 'und winkte Flametti', 'Zitternd und bebend beeilte sie sich', 'grüßte er mit salopp geschwungener Schneidigkeit und blödem Gesichtsausdruck', 'grüßte Flametti souverän', 'nickte Jenny mit ihrem süßesten Lächeln', 'zitterte Lydia mit erfrorener Nase', 'Und jener krausköpfige Herr kam mit dem Knaben durch die Kulisse herein, zitternd und bebend', 'Aber Flametti schüttelte den Kopf', 'und reichte dem Meyer zitternd vor Ergriffenheit die Hand', 'Die Soubrette wandte aufhorchend den Kopf', 'zwinkerte er', 'und parierte mit einem mitleidigen Achselzucken', 'und zitterte', 'und winkte nach rückwärts', 'Frau von Herbeck schüttelte leicht den Kopf', 'Emmy erbebte unter seinem Blicke', 'und sie grüßte mit ebensoviel Anstand als Würde', 'Wie geistreich behandelte er seine von ihm aufgestellten, oft nur zu paradox klingenden Ideen', 'Er entschuldigte sich mit seinem gewöhnlichen satirischen und doch wieder so gutmüthigen Lächeln', 'Er reichte mir die Hand mit seiner alten Herzlichkeit hin', 'Ich drückte dem alten Sommer verstohlen die Hand', 'und winkte', 'Der General winkte noch einmal', 'Er reichte Gustav die Hand', 'drückte sie', 'Daniel reichte dem jungen Manne die Hand', 'und drückte sie herzhaft', 'Sie erwiderte zitternd den Druck', 'wie sie bebte', 'Daniel schüttelte langsam den Kopf', 'Mit herzlichem Händedruck verabschiedeten sich die beiden', 'und nun zitternd, mit halb offenem Munde, die beiden Hände gegen Leonoren ausstreckend', 'Endlich schüttelte er bedächtig den Kopf', 'Das arme Mädchen zitterte und bebte', 'Er drückte die kleinen Hände des hübschen Mädchens', 'Sie winkte uns', 'schüttelte jedoch den Kopf', 'die mir jetzt wohlwollend die Hand zum Abschied reichte', 'bevor er sich mit einem freundlich-gleichmütigen Kopfnicken zurückgezogen', 'denn er gab mir nur noch schnell die Hand', 'winkten wir noch einander zu', 'während Fritz in seiner dünnen rauchgeschwärzten Kleidung wie ein Espenlaub zitterte', 'daß der arme, vor Kälte und überstandener Todesangst zitternde Knabe dort hier in’s warme Nest gehört', 'und sie reichte Irene die Hand', 'Irene schüttelte den Kopf', 'Wilhelmine reichte nochmals ihren Gästen die Hand', 'und dem fremden Herrn ein Händchen gegeben hatten', 'Sie bebte zusammen', 'und reichte ihm die Hand', 'Er küßte lächelnd ihre Stirne', 'die ganze Zeit über zitterte er unter dem Kanapee', 'sagte, allerdings zitternd und unüberlegt', 'viertelstundenlang schüttelte er langsam den Kopf', 'trotz der beschwörend erhobenen Hände der Mutter', 'der mittlere Zimmerherr lächelte erst einmal kopfschüttelnd seinen Freunden zu', 'aber das starke Nicken seines wie haltlosen Kopfes zeigte', 'Die Bedienerin stand lächelnd in der Tür', 'wehrte dies mit ausgestreckter Hand entschieden ab', 'die Erde erbebte', 'man winkt ihm', 'wie er ihm feierlich die Hand schüttelte', 'und mit einem kurzen, zerstreuten Kopfnicken sogleich wieder verschwand', 'Die beiden Männer reichten sich die Hände', 'indem er dazu bedenklich den Kopf hin und her wiegte', 'Hildegard schüttelte den Kopf', 'Frau Wallner schüttelte den Kopf', 'und drückte Hildegards Hand', 'Frau von Werdern nickte gedankenvoll mit dem Kopfe', 'Hildegard drückte ihr die Hand', 'und schüttelte ihr die Hand', 'und legte ihr die Hand auf die Schulter', 'Die schöne Frau machte eine abwehrende Handbewegung', 'die Hildegard freundlich die Hand drückte', 'Elvira winkte der Kellnerin', 'Hildegard schüttelte den Kopf', 'aus der eine Dame Elvira gnädig zunickte', 'Die Lehrerin verneigte sich tief', 'Elvira zuckte die Schultern', 'Hildegard schüttelte den Kopf', 'Dann reichte sie Elviren die Hand hin', 'Elvira wiegte den Kopf', 'Elvira rieb sich vor Vergnügen die Hände', 'Fräulein Schulze machte eine Handbewegung', 'und winkte eine Droschke herbei', 'ergriff ihre Hand', 'Er wiegte nachdenklich den Kopf', 'zitterte', 'und schüttelte mir die Hand', 'Heiter schüttelte er mir die Hand', 'und winkte mir noch', 'Aber mein Besucher schüttelte den Kopf', 'und schüttelte mir die Hand', 'schüttelte er ein wenig den Kopf', 'Professor Müller schüttelte den Kopf', 'und Roland schüttelte den Kopf', 'Ich hob die Schultern', 'Miß Mason schüttelte den Kopf über den Betrieb in der Klinik', 'Sie wandte sich kopfschüttelnd mir zu', 'und Walter winkte Fred', 'und senkte den Kopf auf die Seite', 'Er machte auch eine abwehrende Handbewegung', 'Da faßte er meine Hand', 'küßte sie', 'Die Fabianen nickte heftig mit dem Kopfe', 'Und der Fabianen reichte sie die Hand', 'und sie an der Schulter rüttelte', 'Der Förster wiegte bedenklich den Kopf', 'desto mehr mit den Köpfen schüttelte', 'bebte er, bis in seine innersten Poren', 'scheu und zitternd stand sie noch einen vollen Schritt von dem Tisch entfernt', 'schüttelte seinen kleinen Kopf und die Frackschöße', 'winkte und nickte nach der Chiffonniere hinüber', 'Der frische Morgenwind schnellt den Thau von den bebenden Halmen', 'Dann schüttelte sie heftig den Kopf', 'Eines Mittags aber hatte sie mein Grußlächeln so fröhlich erwiedert', 'und sie lächelte zuversichtlich, stolz', 'Wieder mußte ich lächeln', 'aber sie schüttelte doch den Kopf', 'so grüßten wir uns mit dem erkennenden Lächeln', 'Sie zuckte mit ihrem gewohnten liebenswürdigen Lächeln die Achseln', 'wieder drückte sie meine Hand', 'und lächelte ernsthaft', 'und wir drückten uns noch einmal fest die Hände', 'schüttelte meine Hand und war verschwunden', 'Der Beamte schüttelte den Kopf', 'Ich drückte die hartgearbeitete gute Hand der Frau', 'Frau Laubi nickte ernsthaft', 'Wieder kam das vielsagende Achselzucken', 'Ich drückte dem jungen Manne fest die Hand', 'Und herzhaft erwiederte er den Druck', 'Ich habe mit Begeisterung mein Handgelübde abgelegt', 'und meine Hand zittert', 'Da kopfschüttelte er', 'schüttelte sie den Kopf', 'mein Herz bebte', 'Kohn hörte kopfschüttelnd zu', 'Mit diesen Worten setzte er den Hut auf den Kopf', 'Aber wie sie s mir dann auf den Kopf zusagten', 'um dieser die Hand zu küssen', 'die Mama, die, von der Steintreppe her, mit ihrem Taschentuche winkte', 'und gab ihm einen Kuß auf die linke Backe', 'ja dem Lieblinge zur Besiegelung des Einverständnisses einen Kuß auf die Stirn gegeben', 'Effi winkte mit dem Taschentuch', 'und küßte Effi', 'und küßte ihre beiden Hände', 'grüßte Effi vom Koupee aus', 'und reichte Innstetten die Hand', 'Effi reichte dem verlegen Eintretenden die Hand', 'die dieser mit einem gewissen Ungestüm küßte', 'Effi gab ihm die Hand', 'nach wiederholtem Handkuß', 'Innstetten nahm ihre Hand', 'schüttelte den Kopf', 'Kruse wiegte den Kopf hin und her', 'Ein Zittern überkam sie', 'und überdeckte sie mit heißen Küssen', 'und zitterte', 'und sie reichte ihm die Hand', 'und ihr ganzer zarter Körper zitterte', 'Noch ein Händedruck', 'Sie zitterte vor Erregung', 'Effi schüttelte den Kopf', 'Effi schüttelte den Kopf', 'Zugleich deutete Crampas durch eine Handbewegung an', 'und küßte ihm die Hand', 'und ihr die Hand reichte', 'Roswitha gab dem Kinde einen Kuß', 'und gab ihm einen Kuß auf die Stirn', 'und gab ihr einen Kuß auf die Stirn', 'Wiesike wiegte den Kopf langsam hin und her', 'schüttelte den Kopf langsam hin und her', 'Beglückt und verständnißinnig reichten sich Bernhard und Reinhold die Hand', 'Tief ergriffen reichte Reinhold dem Freunde die Hand', 'ebenfalls innig bewegt die Hand des Freundes drückend', 'Der Schimmel schüttelte den Kopf', 'schüttelte den Kopf', 'Nach langem Hinstarren nickte er wohl langsam mit dem Kopfe', 'Der Alte schüttelte den Kopf', 'Elke schüttelte den Kopf', 'Aber sie schüttelte den Kopf', 'er aber kehrte den Kopf ab', 'aber sie schüttelte nur den Kopf', 'Sie schüttelte den Kopf', 'Aber der Alte schüttelte den Kopf', 'Der Oberdeichgraf aber reichte dem Mädchen seine Hand', 'Sie drückte ihm die Hand', 'Hauke schüttelte den Kopf', 'nachdem der Junge ihm nachdrücklich darauf die Hand geboten hatte', 'Der Junge schüttelte den Kopf', 'aber sie schüttelte den Kopf', 'Hauke schüttelte den Kopf', 'und ging kopfnickend aus dem Zimmer', 'und da das Kind wie nickend das Köpfchen senkte', 'begann sie mit dem Kopfe zu schütteln', 'Aber Hauke schüttelte den Kopf', 'Er schüttelte den Kopf', 'der die Hand schwenkte']





In [19]:
# Nice! This one works!
# <character_sound></character_sound>
# If with automatically generated annotations, pay attention to the "loudness="\d" because it is not part of the automatically generated annotations from NEISS NTEE.

import os
import re

def process_xml_files(folder_path, strings):
    # Iterate over each xml file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(folder_path, filename)
            # Read the text file
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            # Iterate over strings to find and delete the regex matches
            for string in strings:
                # Define the regex pattern to match the combination
                regex_pattern = fr'<(?:ambient|character)_sound>\s*{re.escape(string)}\s*</(?:ambient|character)_sound>'

                # Find and delete the regex match from the text
                text = re.sub(regex_pattern, string, text)

            # Write the modified text back to the file
            with open(xml_file_path, 'w', encoding='utf-8') as file:
                file.write(text)

# Execute the function
process_xml_files(folder_path, strings)

In [20]:
print("Frequent false positive cleaning finished.")

Frequent false positive cleaning finished.


## Sound Event Span Extraction

In [21]:
#This code extracts the sound event spans and stores them in two separate lists according to their ambient or character sound classification. 

#import os
#import xml.etree.ElementTree as ET

#the following function creates empty lists, iterates over the xml elements extracting the content between the elements ambient_sound and character_sound to store them in the empty list, sorted by xml text file using the list.append and extend commands.

def extract_sound_spans(xml_content):
    ambient_sound_spans = []
    character_sound_spans = []
    root = ET.fromstring(xml_content)

    ambient_sound_text = ""
    character_sound_text = ""
    
    for elem in root.iter():
        if elem.tag.endswith('ambient_sound'):
            ambient_sound_text = elem.text.strip()
            ambient_sound_spans.append(ambient_sound_text)
        elif elem.tag.endswith('character_sound'):
            character_sound_text = elem.text.strip()
            character_sound_spans.append(character_sound_text)
    return ambient_sound_spans, character_sound_spans

def process_xml_file(filepath):
    ambient_sound_spans_list = []
    character_sound_spans_list = []
    with open(filepath, 'r', encoding='utf-8') as file:
        xml_content = file.read()
        ambient_sound_spans, character_sound_spans = extract_sound_spans(xml_content)
        ambient_sound_spans_list.extend(ambient_sound_spans)
        character_sound_spans_list.extend(character_sound_spans)
    return ambient_sound_spans_list, character_sound_spans_list

def process_folder(folder_path):
    sound_spans_per_file = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            filepath = os.path.join(folder_path, filename)
            ambient_sound_spans_list, character_sound_spans_list = process_xml_file(filepath)
            sound_spans_per_file[filename] = {'ambient_sound_spans': ambient_sound_spans_list, 
                                              'character_sound_spans': character_sound_spans_list}
    return sound_spans_per_file


sound_spans_per_file = process_folder(folder_path)

for filename, sound_spans in sound_spans_per_file.items():
    print("File:", filename)
    print("Ambient Sound Spans:", sound_spans['ambient_sound_spans'])
    print("Character Sound Spans:", sound_spans['character_sound_spans'])
    print()


File: Ring_Max_Vom_alten_Heim.xml
Ambient Sound Spans: []
Character Sound Spans: ['Während er sprach', 'mit denen er sich unterhielt', 'Er sprach mit ihnen wie ein älterer Freund mit seinen jüngeren Collegen', 'Beide Aerzte sprachen zuerst von dem vorliegenden Falle', 'fragte er ihn in dem ihm eigenen salbungsvollen Tone', 'dieser Frage', 'antwortete er schmunzelnd', 'Dabei klopfte der lose Spötter gutmüthig seinem Collegen auf die Schulter', 'und verschrieben mehr zur Beruhigung der Angehörigen eine Medicin, ohne sich die geringste Wirkung zu versprechen', 'so versprach Heim', 'fügte er mit ernstem Gesichte hinzu', 'Der furchtsame Patient gelobte den strengsten Gehorsam', 'Heim erklärte', 'und überhäufte seinen Collegen mit Vorwürfen', 'rief er erschrocken aus', 'antwortete der bescheidene Heim mit einer fast kindlichen Einfalt', 'schrie die Frau', 'einer so kläglichen Aufforderung', 'Heim erkundigte sich theilnehmend nicht nur nach dem Befinden, sondern auch nach den näheren Umstände

In [22]:
# Counter of the sound events separately for each class and summed up.
def count_sound_events(sound_spans_per_file):
    ambient_sound_count = 0
    character_sound_count = 0
    total_sound_count = 0
    
    for sound_spans in sound_spans_per_file.values():
        ambient_sound_count += len(sound_spans['ambient_sound_spans'])
        character_sound_count += len(sound_spans['character_sound_spans'])
    
    total_sound_count = ambient_sound_count + character_sound_count
    
    return ambient_sound_count, character_sound_count, total_sound_count

# Call the function to count sound events
ambient_sound_count, character_sound_count, total_sound_count = count_sound_events(sound_spans_per_file)

# Print the results
print("Number of Ambient Sound Events:", ambient_sound_count)
print("Number of Character Sound Events:", character_sound_count)
print("Total Number of Sound Events:", total_sound_count)


Number of Ambient Sound Events: 2894
Number of Character Sound Events: 18217
Total Number of Sound Events: 21111


In [25]:
# Write the output to a CSV file
#import csv
#output_file = '/Users/sguhr/Desktop/Diss_notebooks/ner_prediction_sicherheitskopie_20240505_15h/20240501_Subcorpus_1848-55_predicted_for_loudness/20240509_sound_spans_output_Subcorpus_1848-55_predicted_test.csv'

output_file = output_file

with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['File', 'Ambient Sound Spans', 'Character Sound Spans']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for filename, sound_spans in sound_spans_per_file.items():
        writer.writerow({'File': filename,
                         'Ambient Sound Spans': sound_spans['ambient_sound_spans'],
                         'Character Sound Spans': sound_spans['character_sound_spans']})

In [26]:
print("Sound event extraction finished and saved to output csv file.")

Sound event extraction finished and saved to output csv file.


## Open the saved table as a pandas dataframe

The next step is to open the csv file as a pandas data frame.

In [33]:
# Import 
#import os
#import csv
#import pandas as pd
#import regex as re
#from pathlib import Path
#from collections import Counter

#csv_file_path = '/Users/sguhr/Desktop/Diss_notebooks/ner_prediction_sicherheitskopie_20240505_15h/20240501_Subcorpus_1848-55_predicted_for_loudness/revised/20240509_sound_spans_output_Resel_Verwandlung_loudness_new.csv'

#Indicate the path to the source csv file
csv_file_path = output_file

# Read the CSV file into a Pandas DataFrame
diss_corpus_annotations = pd.read_csv(csv_file_path)

# Display the DataFrame
print(diss_corpus_annotations.head())

                                                File  \
0                        Ring_Max_Vom_alten_Heim.xml   
1                                  Ring_Max_Sand.xml   
2  Wildermuth_Ottilie_Onkel_Gottliebs_Jugendliebe...   
3  Rasch_Gustav_Ein_Schneesturm_auf_den_Hohen_Hei...   
4                      Storch_Ludwig_Das_Loechle.xml   

                                 Ambient Sound Spans  \
0                                                 []   
1  ['Beide stießen zuerst mit den klingenden Röme...   
2  ['das Kindlein,', 'Jubel begrüßt', 'Da erschol...   
3  ['Gebirgsbewohner', 'Tauernhause, Musik und Sc...   
4  ['und die Spitzen dampfen nur noch leise drübe...   

                               Character Sound Spans  
0  ['Während er sprach', 'mit denen er sich unter...  
1  ['je länger sie mitsammen sprachen', 'wenn das...  
2  ['begann die kleine Hedwig bedenklich', 'rief ...  
3  ['und mich aufforderte', 'und hatte mir in ein...  
4  ['wie Ritter Lang in seiner Geschichte des Für..

The following function cleans the string representation of the lists.
This code reads the CSV file, cleans the string representations of lists in each row, and then extracts the file name, ambient sound spans, and character sound spans. Finally, it prints the extracted data for verification.

In [34]:
#import csv


# Define a function to clean the string representation of lists
def clean_list_string(list_string):
    # Remove leading and trailing whitespace
    cleaned = list_string.strip()
    # Remove leading and trailing square brackets
    cleaned = cleaned.strip("[]")
    # Split the string into a list using comma as separator
    cleaned_list = cleaned.split(", ")
    # Remove leading and trailing quotes from each element in the list
    cleaned_list = [element.strip("'\"") for element in cleaned_list]
    return cleaned_list

# Define a function to process each row of the CSV
def process_csv_row(row):
    file_name = row['File']
    ambient_sound_spans = clean_list_string(row['Ambient Sound Spans'])
    character_sound_spans = clean_list_string(row['Character Sound Spans'])
    return file_name, ambient_sound_spans, character_sound_spans

# Read the CSV file and process each row
#csv_file_path = 'your_csv_file.csv'  # Replace 'your_csv_file.csv' with the path to your CSV file
sound_data = []
with open(csv_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sound_data.append(process_csv_row(row))

# Print the extracted data for verification
for file_name, ambient_sound_spans, character_sound_spans in sound_data:
    print("File:", file_name)
    print("Ambient Sound Spans:", ambient_sound_spans)
    print("Character Sound Spans:", character_sound_spans)
    print()


File: Ring_Max_Vom_alten_Heim.xml
Ambient Sound Spans: ['']
Character Sound Spans: ['Während er sprach', 'mit denen er sich unterhielt', 'Er sprach mit ihnen wie ein älterer Freund mit seinen jüngeren Collegen', 'Beide Aerzte sprachen zuerst von dem vorliegenden Falle', 'fragte er ihn in dem ihm eigenen salbungsvollen Tone', 'dieser Frage', 'antwortete er schmunzelnd', 'Dabei klopfte der lose Spötter gutmüthig seinem Collegen auf die Schulter', 'und verschrieben mehr zur Beruhigung der Angehörigen eine Medicin', 'ohne sich die geringste Wirkung zu versprechen', 'so versprach Heim', 'fügte er mit ernstem Gesichte hinzu', 'Der furchtsame Patient gelobte den strengsten Gehorsam', 'Heim erklärte', 'und überhäufte seinen Collegen mit Vorwürfen', 'rief er erschrocken aus', 'antwortete der bescheidene Heim mit einer fast kindlichen Einfalt', 'schrie die Frau', 'einer so kläglichen Aufforderung', 'Heim erkundigte sich theilnehmend nicht nur nach dem Befinden', 'sondern auch nach den näheren Um

Distribute the data from the table to dataframe columns.

In [35]:
# prepare the data frame by distributing the data over named columns, one with the sound event span, one with the assigned sound class, one with the file name the sound event spans belong to
# import pandas as pd
# import csv

# Define a function to convert the list of spans to a DataFrame
def spans_to_dataframe(spans, annotation_class, filename):
    df = pd.DataFrame({'annotation_span': spans, 'annotation_class': annotation_class, 'filename': filename})
    return df

# Define a function to process each row of the CSV
def process_csv_row(row):
    file_name = row['File']
    ambient_sound_spans = clean_list_string(row['Ambient Sound Spans'])
    character_sound_spans = clean_list_string(row['Character Sound Spans'])
    
    # Convert spans to DataFrame
    ambient_df = spans_to_dataframe(ambient_sound_spans, 'ambient_sound', file_name)
    character_df = spans_to_dataframe(character_sound_spans, 'character_sound', file_name)
    
    return ambient_df, character_df

# Read the CSV file and process each row
#csv_file_path = 'your_csv_file.csv'  # Replace 'your_csv_file.csv' with the path to your CSV file
sound_data = []
with open(csv_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        ambient_df, character_df = process_csv_row(row)
        sound_data.append(ambient_df)
        sound_data.append(character_df)

# Concatenate the DataFrame for each row into a single DataFrame
final_df = pd.concat(sound_data, ignore_index=True)

# Print the resulting DataFrame
print(final_df)


                                         annotation_span annotation_class  \
0                                                           ambient_sound   
1                                      Während er sprach  character_sound   
2                           mit denen er sich unterhielt  character_sound   
3      Er sprach mit ihnen wie ein älterer Freund mit...  character_sound   
4      Beide Aerzte sprachen zuerst von dem vorliegen...  character_sound   
...                                                  ...              ...   
22988  Die Stimme der schönen Prinzessin von Kurland ...  character_sound   
22989                        und beantwortete die Fragen  character_sound   
22990                                    Herr Tiedge las  character_sound   
22991    und erntete die Lobsprüche der Gesellschaft ein  character_sound   
22992  und waren im lebhaften Gespräch mit ihnen begr...  character_sound   

                                                filename  
0               

Lemmatize and lowercase a copy of the sound event spans saved to the column "lemmatized_sound_span"
The following code reads the CSV file, processes each row to convert the spans into separate DataFrames for ambient and character sounds, and then concatenates all the DataFrames into a single DataFrame. The resulting DataFrame contains two columns: "annotation_span" and "annotation_class", where each row represents a single span and its corresponding class. Furthermore, it adds the filename of the file the each sound span had been extracted from.

In [36]:
# In the following the sound event spans get prepared for the loudness level labeling.
#import pandas as pd
#import csv
#import spacy

# Load the German medium model
#nlp = spacy.load('de_core_news_md')

# Define a function to lemmatize and lowercase the spans
def lemmatize_spans(spans):
    lemmatized_spans = []
    for span in spans:
        doc = nlp(span)
        lemmatized_span = ' '.join([token.lemma_ for token in doc])
        lemmatized_spans.append(lemmatized_span.lower())  # Convert to lowercase
    return lemmatized_spans

# Define a function to convert the list of spans to a DataFrame
def spans_to_dataframe(original_spans, lemmatized_spans, annotation_class, filename):
    df = pd.DataFrame({'filename': filename, 'sound_span': original_spans, 'annotation_class': annotation_class, 'lemmatized_sound_span': lemmatized_spans})
    return df

# Define a function to process each row of the CSV
def process_csv_row(row):
    file_name = row['File']
    ambient_sound_spans = clean_list_string(row['Ambient Sound Spans'])
    character_sound_spans = clean_list_string(row['Character Sound Spans'])
    
    # Lemmatize the spans
    lemmatized_ambient_spans = lemmatize_spans(ambient_sound_spans)
    lemmatized_character_spans = lemmatize_spans(character_sound_spans)
    
    # Convert spans to DataFrame
    ambient_df = spans_to_dataframe(ambient_sound_spans, lemmatized_ambient_spans, 'ambient_sound', file_name)
    character_df = spans_to_dataframe(character_sound_spans, lemmatized_character_spans, 'character_sound', file_name)
    
    return ambient_df, character_df

# Read the CSV file and process each row
#csv_file_path = 'your_csv_file.csv'  # Replace 'your_csv_file.csv' with the path to your CSV file
sound_data = []
with open(csv_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        ambient_df, character_df = process_csv_row(row)
        sound_data.append(ambient_df)
        sound_data.append(character_df)

# Concatenate the DataFrame for each row into a single DataFrame
final_df = pd.concat(sound_data, ignore_index=True)

# Print the resulting DataFrame
print(final_df)


                                                filename  \
0                            Ring_Max_Vom_alten_Heim.xml   
1                            Ring_Max_Vom_alten_Heim.xml   
2                            Ring_Max_Vom_alten_Heim.xml   
3                            Ring_Max_Vom_alten_Heim.xml   
4                            Ring_Max_Vom_alten_Heim.xml   
...                                                  ...   
22988  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22989  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22990  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22991  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22992  von_Sternberg_Alexander_Ein_literarischer_Aben...   

                                              sound_span annotation_class  \
0                                                           ambient_sound   
1                                      Während er sprach  character_sound   
2                           mit denen er sich un

Optionally, add part of speech tagging. 

In [39]:
#import pandas as pd
#import csv
#import spacy

print(This cell takes much time and can be skipped.)

skip #comment out if you want to execute this cell.

# Load the German medium model
#nlp = spacy.load('de_core_news_md')

# Define a function to lemmatize and lowercase the spans
def lemmatize_spans(spans):
    lemmatized_spans = []
    for span in spans:
        doc = nlp(span)
        lemmatized_span = ' '.join([token.lemma_ for token in doc])
        lemmatized_spans.append(lemmatized_span.lower())  # Convert to lowercase
    return lemmatized_spans

# Define a function to apply POS tagging to the spans
def pos_tag_spans(spans):
    pos_tagged_spans = []
    for span in spans:
        doc = nlp(span)
        pos_tags = ' '.join([token.pos_ for token in doc])
        pos_tagged_spans.append(pos_tags)
    return pos_tagged_spans

# Define a function to convert the list of spans to a DataFrame
def spans_to_dataframe(original_spans, lemmatized_spans, pos_tagged_spans, annotation_class, filename):
    df = pd.DataFrame({
        'filename': filename,
        'sound_span': original_spans,
        'annotation_class': annotation_class,
        'lemmatized_sound_span': lemmatized_spans,
        'pos_tagged_spans': pos_tagged_spans  # Add the new column for POS tagged spans
    })
    return df

# Define a function to process each row of the CSV
def process_csv_row(row):
    file_name = row['File']
    ambient_sound_spans = clean_list_string(row['Ambient Sound Spans'])
    character_sound_spans = clean_list_string(row['Character Sound Spans'])
    
    # Lemmatize the spans
    lemmatized_ambient_spans = lemmatize_spans(ambient_sound_spans)
    lemmatized_character_spans = lemmatize_spans(character_sound_spans)
    
    # Apply POS tagging to the spans
    pos_tagged_ambient_spans = pos_tag_spans(ambient_sound_spans)
    pos_tagged_character_spans = pos_tag_spans(character_sound_spans)
    
    # Convert spans to DataFrame
    ambient_df = spans_to_dataframe(
        ambient_sound_spans, 
        lemmatized_ambient_spans, 
        pos_tagged_ambient_spans, 
        'ambient_sound', 
        file_name
    )
    character_df = spans_to_dataframe(
        character_sound_spans, 
        lemmatized_character_spans, 
        pos_tagged_character_spans, 
        'character_sound', 
        file_name
    )
    
    return ambient_df, character_df

# Read the CSV file and process each row
# csv_file_path = 'your_csv_file.csv'  # Replace 'your_csv_file.csv' with the path to your CSV file
sound_data = []
with open(csv_file_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        ambient_df, character_df = process_csv_row(row)
        sound_data.append(ambient_df)
        sound_data.append(character_df)

# Concatenate the DataFrame for each row into a single DataFrame
final_df = pd.concat(sound_data, ignore_index=True)

# Print the resulting DataFrame
print(final_df)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3478887660.py, line 4)

In [40]:
#show the data frame
final_df

,filename,sound_span,annotation_class,lemmatized_sound_span
0,Ring_Max_Vom_alten_Heim.xml,,ambient_sound,
1,Ring_Max_Vom_alten_Heim.xml,Während er sprach,character_sound,während er sprechen
2,Ring_Max_Vom_alten_Heim.xml,mit denen er sich unterhielt,character_sound,mit der er sich unterhalten
3,Ring_Max_Vom_alten_Heim.xml,Er sprach mit ihnen wie ein älterer Freund mit...,character_sound,er sprechen mit ihnen wie ein alt freund mit s...
4,Ring_Max_Vom_alten_Heim.xml,Beide Aerzte sprachen zuerst von dem vorliegen...,character_sound,beide aerzter sprechen zuerst von der vorliege...
...,...,...,...,...
22988,von_Sternberg_Alexander_Ein_literarischer_Aben...,Die Stimme der schönen Prinzessin von Kurland ...,character_sound,der stimme der schön prinzessin von kurland ha...
22989,von_Sternberg_Alexander_Ein_literarischer_Aben...,und beantwortete die Fragen,character_sound,und beantworten der frage
22990,von_Sternberg_Alexander_Ein_literarischer_Aben...,Herr Tiedge las,character_sound,herr tiedge lesen
22991,von_Sternberg_Alexander_Ein_literarischer_Aben...,und erntete die Lobsprüche der Gesellschaft ein,character_sound,und ernten der lobsprüche der gesellschaft ein


In [ ]:
print("The dataframe is prepared for automatized loudness level annotation.")

Given a sound event annotated corpus prepared as a csv file with extracted sound annotations, the following script detects sound words in the spans that match a loudness level dictionary after lemmatizing and lowercasing the sound event spans, and writes the loudness values as attributes back to the XML files. 

## Loudness Level Labeling

The following code defines a function find_sound_words that takes a string of text as input, splits it into words, and then checks if each word is a key in the sound dictionary. If a word is found in the dictionary, it adds it to a list. Finally, it adds a new column to your dataframe called 'found_sound_words', which contains lists of found sound words for each lemmatized sound_span.

In [43]:
# In the following, I provide the loudness level dictionary containing key-value pairs from sound words as keys and loudness level as values.

loudness_dict = {'abbringen': 3, 'abgeschiedenheit': 1, 'ablehnen': 3, 'abmachen': 3, 'abmahnung': 3, 'absprechen': 3, 'aburteilen': 3, 'accorde': 4, 'ächz': 4, 'ächzen': 4, 'ächzend': 4, 'aechzen': 4, 'anbieten': 3, 'anbrüllen': 4, 'andacht': 0, 'andeuten': 3, 'andichten': 3, 'anempfehlen': 3, 'anempfehlung': 3, 'anfechten': 4, 'angeben': 3, 'anhören': 0, 'anklagen': 3, 'anlächeln': 0, 'anpöbeln': 4, 'anprangern': 4, 'anpreisen': 3, 'anraten': 3, 'anreden': 3, 'anrufen': 4, 'anschreien': 4, 'anschwärzen': 3, 'ansprechen': 3, 'antworteen': 3, 'antworten': 3, 'anweisung': 3, 'applaudieren': 4, 'applaus': 4, 'atem': 2, 'atemholen': 1, 'atemzug': 1, 'atemzüge': 1, 'atemzügen': 1, 'athem': 1, 'athemholen': 1, 'athemzug': 1, 'athmen': 1, 'athmend': 1, 'athmet': 1, 'atme': 1, 'atmeen': 1, 'atmen': 1, 'atmend': 1, 'atmeter': 1, 'aufathmen': 1, 'aufathmend': 1, 'aufatmen': 1, 'aufatmend': 1, 'auffordern': 3, 'aufhorchen': 1, 'aufruhr': 4, 'aufschluchzen': 2, 'aufschreien': 4, 'aufseufzen': 3, 'aufseufzend': 3, 'aufspiele': 3, 'aufstassen': 4, 'aufstöhnend': 3, 'auftrag': 3, 'auftreten': 3, 'aufwartung': 3, 'aufzuatmen': 1, 'auseinandersetzung': 4, 'auslachen': 4, 'auspacken': 3, 'auspfeifen': 4, 'ausrufen': 4, 'ausschreit': 4, 'äußern': 3, 'aussprechen': 3, 'ausstoßen': 4, 'ausstoßend': 3, 'auszustoßen': 4, 'axthieb': 5, 'bahnhofslärm': 4, 'ballern': 4, 'barsch': 4, 'baßtrompete': 4, 'baulärm': 5, 'beachten': 3, 'beanstanden': 3, 'beantworten': 3, 'bedauern': 3, 'befrage': 3, 'befriedigen': 3, 'beglückwünschen': 3, 'begrüßen': 3, 'begrüßungsformalität': 3, 'behaupten': 3, 'beherrschen': 3, 'beichten': 3, 'beifall': 4, 'beifallsäußerung': 4, 'beipflichten': 3, 'bejahen': 3, 'bejahend': 3, 'bejubeln': 4, 'bekenntniß': 3, 'beklagen': 3, 'beklatschen': 4, 'beklommen': 0, 'bekräftigen': 4, 'belächeln': 1, 'belauschen': 1, 'belehren': 3, 'bellen': 4, 'bellend': 4, 'bemäkeln': 3, 'bemerken': 3, 'beraten': 3, 'bereden': 3, 'berichten': 3, 'berichtigen': 3, 'bersten': 4, 'beruhigen': 3, 'beschämt': 0, 'bescheidener': 3, 'beschimpfen': 4, 'beschimpft': 4, 'beschuldigen': 4, 'beschuss': 5, 'beschwatzen': 3, 'beschwichtigen': 3, 'beschwören': 3, 'besingen': 4, 'besprechen': 3, 'bestätigen': 3, 'bestimmen': 3, 'bestreiten': 4, 'bestürzt': 3, 'beteuern': 3, 'betonen': 4, 'bewilligen': 3, 'bezichtigen': 4, 'bitter': 4, 'bitten': 3, 'blasen': 4, 'blasend': 4, 'blasmusik': 4, 'blöken': 4, 'blökend': 4, 'bloßstellen': 3, 'bombe': 5, 'bösartigen': 4, 'brandung': 4, 'brausen': 4, 'brauste': 4, 'brise': 2, 'brüllen': 4, 'brüllend': 4, 'brülln': 4, 'brüllte': 4, 'brummen': 2, 'brüskieren': 4, 'buchstabierte': 3, 'bumm': 5, 'chor': 4, 'dämpfen': 2, 'dampfpfeife': 5, 'danken': 3, 'darbracht': 3, 'declamirende': 4, 'denunzieren': 3, 'detonation': 5, 'diffamieren': 4, 'diskret': 2, 'diskutieren': 4, 'donner': 5, 'donnerklang': 5, 'donnern': 5, 'donnernd': 5, 'donnerschlag': 5, 'drängelen': 4, 'drehen': 3, 'dröhnen': 5, 'dröhnend': 5, 'druckwelle': 5, 'dumpf': 2, 'dumpfen': 1, 'durchbrechen': 3, 'echo': 3, 'echot': 4, 'einatmend': 1, 'eindreangen': 3, 'einflüstern': 2, 'eingeschlafen': 1, 'einholen': 3, 'einladen': 3, 'einlullende': 2, 'einräumen': 3, 'einsam': 0, 'einschlaufen': 1, 'einschlürfend': 2, 'einsilbig': 3, 'einspruch': 4, 'einstimmen': 3, 'einstürzend': 5, 'eintönig': 3, 'einwand': 4, 'einwendung': 4, 'einwilligen': 3, 'einwilligung': 3, 'einwurf': 4, 'empfehlen': 3, 'entfuhr': 3, 'entgegenträumend': 0, 'entgegnen': 3, 'entschuldigend': 3, 'erdröhnen': 5, 'erfahren': 3, 'ergänzen': 3, 'erhobener': 4, 'erklären': 3, 'erklingen': 3, 'erlassen': 3, 'erlauben': 3, 'ermahnung': 4, 'erniedrigen': 4, 'ernst': 3, 'erröten': 1, 'erröthen': 1, 'erschallen': 4, 'erscholl': 4, 'ersticken': 2, 'erstickt': 2, 'ersuchen': 3, 'ertönen': 3, 'erwideren': 3, 'erwidern': 3, 'erwiederen': 3, 'erwiedern': 3, 'erzählen': 3, 'erzählstn': 3, 'erzählung': 3, 'exkommunizieren': 3, 'explodieren': 5, 'explosion': 5, 'fanfare': 4, 'fassen': 3, 'fauchen': 4, 'feiern': 4, 'feuern': 5, 'fiepen': 4, 'flatternd': 3, 'flehen': 3, 'flennen': 4, 'fließen': 3, 'floss': 3, 'flöten': 4, 'fluchen': 4, 'fluchn': 4, 'fluglärm': 5, 'flüsteren': 2, 'flüstern': 2, 'flüsternd': 2, 'flüstert': 2, 'flüsterte': 2, 'föppelt': 3, 'forte': 4, 'fortfahren': 3, 'fortfuhr': 3, 'fortissimo': 5, 'fragen': 3, 'fragt': 3, 'freundlicher': 3, 'friede': 0, 'friedhofsruhe': 0, 'frohlocken': 3, 'frug': 3, 'fügen': 3, 'fußtritt': 3, 'gackern': 4, 'gähnen': 1, 'galoppiern': 4, 'gardinenpredigt': 4, 'geäußert': 3, 'gebärde': 1, 'gebärden': 1, 'gebärdensprache': 1, 'gebell': 4, 'gebet': 0, 'gebetsglocke': 4, 'gebrüll': 4, 'gedämpft': 2, 'gedröhne': 5, 'gedudel': 4, 'geflüstert': 2, 'gegenrede': 4, 'gegenstimme': 4, 'geheim': 2, 'geheul': 4, 'gehupe': 5, 'gejammer': 4, 'gejohle': 4, 'geklapper': 4, 'geklirr': 4, 'geknatter': 4, 'gekrache': 5, 'gelächter': 4, 'geläute': 4, 'gellen': 4, 'gellend': 4, 'gellt': 4, 'geloben': 3, 'gemurmel': 2, 'genehmigen': 3, 'geplapper': 4, 'geplätscher': 4, 'gepolter': 5, 'geprassel': 4, 'gerassel': 4, 'geräusch': 3, 'geräusche': 3, 'geräuschlos': 1, 'geräuschvoll': 4, 'gesang': 4, 'geschnatter': 4, 'geschrei': 4, 'geschrieen': 4, 'geschrien': 4, 'gespräch': 3, 'gesprächig': 3, 'gesungene': 4, 'getadelt': 4, 'getöse': 4, 'gewitter': 4, 'gezeter': 4, 'glockenläuten': 5, 'glockenton': 4, 'glucksen': 2, 'grabesstille': 0, 'granate': 5, 'gratulieren': 3, 'gröhlen': 4, 'grollen': 4, 'grübelen': 0, 'grunzen': 4, 'gruß': 3, 'grüßen': 3, 'gurgeln': 3, 'gurgelnd': 3, 'gurgelton': 2, 'gurren': 3, 'halblaut': 3, 'halbleisen': 2, 'halbleise': 2, 'hall': 4, 'hallen': 4, 'hämmer': 5, 'hämmern': 5, 'hammerschlag': 5, 'händeklatschen': 4, 'handschlag': 3, 'hauchen': 1, 'hauen': 4, 'hehehehehemeh': 4, 'heiben': 4, 'heiser': 2, 'heiseren': 2, 'hellhörig': 1, 'herauslach': 4, 'hervorheben': 4, 'hervorrufen': 4, 'herzschlag': 1, 'hetzen': 4, 'heulen': 4, 'heulend': 4, 'heuln': 4, 'heulte': 4, 'hieb': 4, 'hilferuf': 4, 'hinzufügen': 3, 'hinzusetzen': 3, 'ho-ho': 4, 'höhnen': 4, 'höhnisch': 4, 'höhnisches': 4, 'höllenlärm': 5, 'hörbar': 2, 'horchen': 1, 'hören': 1, 'hörte': 1, 'hufe': 4, 'hufschlag': 4, 'huldigen': 3, 'hülfe': 3, 'hupen': 5, 'hurra': 4, 'huschen': 3, 'hüstelen': 2, 'husten': 3, 'hymne': 4, 'iah': 3, 'ignorieren': 1, 'inbrünstig': 4, 'intoniert': 4, 'isarrausch': 4, 'jammeren': 3, 'jammern': 3, 'jammernd': 3, 'jauchzen': 4, 'jauchzer': 4, 'jaulen': 4, 'jodeln': 4, 'johlen': 4, 'johlten': 4, 'jubeln': 4, 'kalt': 3, 'karikieren': 3, 'keifen': 4, 'keuchen': 3, 'keuchend': 3, 'kicheren': 3, 'kichern': 3, 'kichernd': 3, 'kikeriki': 4, 'kinderlärm': 4, 'kläffen': 4, 'klage': 3, 'klageenden': 3, 'klagelaut': 4, 'klagen': 3, 'klang': 3, 'klänge': 3, 'klangvoll': 3, 'klappern': 4, 'klatschen': 4, 'klatschend': 4, 'kleinlaut': 2, 'klicken': 2, 'klingeln': 4, 'klingelzeichen': 3, 'klingen': 4, 'klingend': 3, 'klingender': 3, 'klirre': 4, 'klirren': 4, 'klirrend': 4, 'klopfen': 4, 'klopfte': 4, 'knabenstimm': 3, 'knacken': 2, 'knall': 5, 'knallen': 5, 'knarren': 4, 'knarrend': 4, 'knatteren': 4, 'knattern': 4, 'knirschen': 2, 'knirschend': 2, 'knistern': 2, 'knurren': 4, 'knurrn': 4, 'kollern': 4, 'kommandieren': 4, 'kommandorufe': 4, 'kommandoworte': 4, 'konstatieren': 3, 'kopfnicken': 1, 'kopfschütteln': 1, 'krach': 4, 'krachen': 5, 'krachend': 5, 'krächzen': 4, 'krächzend': 4, 'krähen': 4, 'kratzen': 3, 'kreischen': 4, 'kreischend': 4, 'kreischte': 4, 'kritisieren': 3, 'kuckucksruf': 3, 'küßen': 2, 'lächelen': 1, 'lächelt': 1, 'lachen': 4, 'lachend': 4, 'lacht': 4, 'lachten': 4, 'laden': 3, 'lallen': 4, 'langsamer': 3, 'lärm': 4, 'lärmbelästigung': 4, 'lärmen': 4, 'lärmend': 4, 'lästern': 3, 'lauschen': 1, 'laut': 4, 'läuten': 4, 'lautlos': 0, 'lautlose': 0, 'lautlosigkeit': 0, 'leise': 2, 'leiser': 2, 'lesen': 3, 'leseprobe': 3, 'leutseliger': 3, 'liedchen': 4, 'lispelen': 2, 'loben': 3, 'lobpreisen': 3, 'luftschöpfen': 1, 'mahnen': 3, 'mäkeln': 3, 'männerschritte': 3, 'markerschütternd': 5, 'marschmusik': 4, 'mäuschenstill': 0, 'meckern': 4, 'meinen': 3, 'miau': 3, 'miauen': 3, 'missklang': 3, 'misston': 3, 'mißton': 4, 'misstönen': 3, 'misstönend': 3, 'mißtrauen': 3, 'mithören': 1, 'mitleidigen': 3, 'mittagsstille': 0, 'mittheilen': 3, 'monoton': 3, 'mosern': 3, 'motorenlärm': 5, 'mucks': 2, 'mucksmäuschenstill': 0, 'murmelen': 2, 'murmeln': 2, 'murmelnd': 2, 'murren': 2, 'musik': 4, 'nachdenken': 0, 'nachdenklich': 0, 'nachdrücklich': 3, 'nachhall': 3, 'nachhallend': 3, 'nachklang': 3, 'nachklingen': 3, 'nachrede': 3, 'nachsinnen': 0, 'nachsprach': 3, 'nachtruhe': 1, 'nachtstille': 0, 'nebelhorn': 5, 'nebengeräusch': 3, 'nebengeräusche': 3, 'nennen': 3, 'niesen': 4, 'nörgeln': 3, 'notschrei': 4, 'öd': 0, 'oede': 0, 'offerieren': 3, 'ohrenbetäubend': 5, 'ohrfeige': 4, 'orgeln': 4, 'paddelnd': 3, 'pauken': 4, 'pause': 0, 'pausierte': 0, 'peitschenartig': 4, 'peitschenhieb': 4, 'pfeifen': 4, 'pfeifend': 4, 'pfeifkonzert': 4, 'pfiffen': 4, 'pianissimo': 2, 'piano': 2, 'piep': 2, 'piepen': 3, 'piepsen': 2, 'plappern': 3, 'plärren': 4, 'plätschern': 3, 'plätschernd': 3, 'platzen': 4, 'plauderen': 3, 'plaudern': 3, 'plumps': 4, 'pochen': 2, 'polteren': 4, 'poltern': 4, 'polternd': 4, 'präsentieren': 3, 'prasselen': 3, 'prasseln': 3, 'prasselnd': 3, 'predigen': 3, 'preisen': 3, 'prompt': 3, 'pst!': 3, 'puff': 3, 'quaken': 4, 'quengeln': 4, 'quieken': 4, 'quietschen': 4, 'radau': 4, 'rannen': 4, 'rapportieren': 3, 'rascheln': 2, 'raschelnd': 2, 'räsonnieren': 3, 'rassel': 4, 'rasseln': 4, 'rasselnd': 4, 'rasselten': 4, 'rastlos': 3, 'ratschen': 4, 'ratschlag': 3, 'rattern': 4, 'rauh': 3, 'raunen': 3, 'rauschen': 3, 'rauschend': 3, 'räusperen': 3, 'rechtfertigen': 4, 'rede': 3, 'reden': 3, 'redend': 3, 'redestrom': 3, 'resignieren': 3, 'rieseln': 2, 'rieth': 3, 'ringgeräusch': 4, 'röcheln': 2, 'röhren': 4, 'rolln': 4, 'ruck': 4, 'ruf': 4, 'rufen': 4, 'rufenwort': 4, 'rügen': 4, 'ruhe': 1, 'ruhestörung': 4, 'ruhig': 1, 'rumpeln': 4, 'rütteln': 3, 'sagen': 3, 'sagte': 3, 'sagts': 3, 'salutieren': 3, 'salve': 3, 'sanft': 2, 'sang': 4, 'sann': 0, 'satzfragment': 3, 'säuseln': 3, 'sausen': 4, 'sausend': 4, 'schaben': 4, 'schalen': 4, 'schall': 4, 'schalldicht': 1, 'schallen': 4, 'schallend': 4, 'schallwelle': 4, 'schalt': 4, 'schäumen': 3, 'schäumend': 3, 'scheiden': 3, 'schellen': 4, 'schelte': 4, 'schelten': 4, 'schepperen': 4, 'scherzen': 3, 'schied': 3, 'schießen': 5, 'schimpfen': 4, 'schimpfirt': 4, 'schimpfwort': 4, 'schlafe': 1, 'schlägerei': 4, 'schlagwort': 3, 'schleichen': 2, 'schleifen': 4, 'schlief': 1, 'schließen': 3, 'schlotternde': 2, 'schluchze': 3, 'schluchzen': 3, 'schluchzte': 3, 'schlucken': 1, 'schlufen': 1, 'schlummer': 1, 'schlummern': 1, 'schlurren': 3, 'schlurrte': 3, 'schmatzen': 3, 'schmettern': 5, 'schmetternd': 5, 'schnalzen': 3, 'schnarchen': 3, 'schnarren': 3, 'schnattern': 3, 'schnauben': 3, 'schnaufen': 2, 'schnauzen': 4, 'schnüffeln': 2, 'schnuppern': 2, 'schnurren': 2, 'scholl': 4, 'schöpfte': 3, 'schrei': 4, 'schreien': 4, 'schreiend': 4, 'schreit': 4, 'schri': 4, 'schrie': 4, 'schrieen': 4, 'schrien': 4, 'schrill': 5, 'schritt': 2, 'schritte': 2, 'schuss': 5, 'schütten': 3, 'schütternd': 5, 'schwach': 2, 'schwall': 4, 'schwatzen': 3, 'schweigen': 0, 'schweigend': 0, 'schweigsam': 0, 'schwellend': 3, 'schwiegen': 0, 'schwirren': 3, 'schwören': 3, 'segnen': 3, 'seufzen': 3, 'seufzer': 3, 'seufzte': 3, 'signal': 5, 'singen': 4, 'singend': 4, 'sirene': 5, 'sirenengesänge': 4, 'sirren': 4, 'sonntagsstille': 0, 'sonor': 3, 'sorgenschwer': 3, 'spätnachmittagsstille': 0, 'spektakel': 4, 'spiel': 4, 'spott': 4, 'spotten': 4, 'spöttisch': 4, 'spöttischer': 4, 'sprache': 3, 'sprächen': 3, 'sprachklang': 3, 'sprachlos': 0, 'sprachlosigkeit': 0, 'sprechen': 3, 'sprengen': 5, 'sprengung': 5, 'sprichen': 3, 'stammeln': 3, 'stammelnd': 3, 'stampfen': 4, 'sterbend': 2, 'stereoton': 3, 'stieß': 4, 'still': 0, 'stille': 0, 'stillschweigen': 0, 'stimme': 3, 'stimmen': 3, 'stimmengewirr': 4, 'stimmlos': 2, 'stimmung': 3, 'stocken': 0, 'stockend': 0, 'stöhnen': 4, 'stöhnend': 4, 'stöhnte': 4, 'stoßen': 4, 'stotteren': 3, 'strafpredigt': 4, 'strafrede': 4, 'straßenlärm': 4, 'streichelen': 2, 'streiten': 4, 'stumm': 0, 'stummheit': 1, 'sturm': 4, 'sturmgeläute': 4, 'stürmisch': 4, 'sturzbach': 4, 'stürzen': 4, 'summen': 2, 'summn': 2, 'surren': 2, 'sympathisierend': 3, 'tadel': 4, 'tadelen': 3, 'tadeln': 4, 'taktlos': 3, 'taktmäßig': 3, 'tamtam': 4, 'tanzweisen': 4, 'täppischer': 3, 'tätschelen': 2, 'tauschen': 3, 'theilen': 3, 'ticken': 2, 'tierlaut': 3, 'tierlaute': 3, 'tierstimme': 3, 'toast': 4, 'toben': 4, 'tobend': 4, 'todesruhe': 0, 'todtenstill': 0, 'todtenstille': 0, 'tone': 3, 'töne': 4, 'tönen': 4, 'tosen': 4, 'tost': 4, 'totenstill': 0, 'totenstille': 0, 'trampeln': 4, 'tremolieren': 4, 'trillern': 4, 'trommeln': 4, 'trommelschlag': 4, 'trommelwirbel': 4, 'trompet': 4, 'trompeten': 4, 'trompetend': 4, 'trompetenstoß': 4, 'tröpfeln': 2, 'trubel': 4, 'trutzliedl': 4, 'tumult': 4, 'tumultuös': 4, 'tuschelen': 2, 'tuscheln': 2, 'überreden': 3, 'übertönen': 5, 'überzeugen': 3, 'umschlich': 2, 'umstimmen': 3, 'umwerben': 3, 'unartikuliert': 2, 'unerhört': 4, 'unterbrechen': 3, 'unterhalten': 3, 'unterhaltungsmusik': 4, 'unterrichten': 3, 'urteilen': 3, 'verabschieden': 3, 'verfluchen': 4, 'verhallen': 2, 'verhalten': 0, 'verhandeln': 3, 'verharren': 0, 'verhauchen': 1, 'verhöhnen': 4, 'verkehren': 3, 'verkehrslärm': 4, 'verkündet': 4, 'verkündigen': 4, 'verlachen': 4, 'vernehmen': 0, 'verneinen': 3, 'verschweigen': 0, 'versetzen': 3, 'versichern': 3, 'verspotten': 4, 'versprechen': 3, 'verständigen': 3, 'verstummen': 0, 'versunken': 0, 'verteidigen': 4, 'verunglimpfen': 4, 'verurteilen': 3, 'verwundertem': 3, 'verwünschung': 4, 'verzeihung': 3, 'verzerrt': 3, 'vielstimmig': 4, 'vorgesagen': 3, 'vorhalten': 4, 'vorlesen': 3, 'vorschlagen': 3, 'vorsprechen': 3, 'vorstellung': 3, 'vortrag': 4, 'vortragen': 4, 'vorwerfen': 4, 'vorwurf': 4, 'vorwürfe': 4, 'vorwurfsvoll': 4, 'waldesfrieden': 0, 'wau': 4, 'wauwau': 4, 'weinen': 3, 'weinend': 3, 'weinte': 3, 'weisen': 3, 'wellenschlag': 4, 'wettern': 4, 'wetzen': 4, 'widerhall': 3, 'widerhallen': 3, 'widerlegung': 3, 'widerrede': 4, 'widersprechen': 3, 'widerspruch': 4, 'widerwillig': 3, 'wiederholen': 3, 'wiederholt': 3, 'wiederholte': 3, 'wiehern': 4, 'wiehernd': 4, 'willkommen': 3, 'wimmeren': 2, 'wimmern': 2, 'wimmert': 2, 'windgeräusche': 3, 'windstoß': 4, 'winseln': 3, 'wirbeln': 3, 'wisperen': 2, 'wispern': 2, 'wohlklang': 3, 'wollen': 3, 'worgeln': 4, 'wort': 3, 'worte': 3, 'wortlos': 0, 'wünschten': 3, 'würdigen': 3, 'wütend': 4, 'zanken': 4, 'zauderen': 0, 'zeichensprache': 1, 'zerbersten': 4, 'zerbrechen': 4, 'zerknäulten': 3, 'zerplatzen': 4, 'zerspringen': 4, 'zerstreuen': 3, 'zetern': 4, 'zeterte': 4, 'zirpen': 3, 'zischelen': 2, 'zischeln': 2, 'zischen': 2, 'zitteren': 3, 'zittern': 1, 'zögeren': 0, 'zögern': 0, 'zögernd': 3, 'zubilligen': 3, 'zuflüsteren': 2, 'zuflüstern': 2, 'zugeben': 3, 'zugeständnis': 3, 'zugestehen': 3, 'zuhören': 1, 'zujubeln': 4, 'zuprosten': 4, 'zureden': 3, 'zurufen': 4, 'zusage': 3, 'zusagen': 3, 'zusammenstauchen': 4, 'zusammentrommeln': 4, 'zusichern': 3, 'zustimmen': 3, 'zustimmend': 3, 'zuzurufen': 4, 'zwischenruf': 4, 'zwitschern': 3,
}



In [57]:
# During the error analysis of the automated loudness level labeling, I recognized the following missing words in the original loudness level dictionary

loudness_dict_extension = {'hub': 3, 'frug': 3, 'begann': 3, 'beginnen': 3, 'fiel': 3, 'einfallen': 3, 'Antwort': 3, 'geben': 3, 'gegeben': 3, 'fuhr': 3, 'fortfahren': 3, 'wandte': 3, 'wenden': 3, 'setzte': 3, 'hinzusetzen': 3, 'laut': 4, 'leise': 2, 'inbrünstig': 4, 'Wort': 3,  'Zahn': 0, 'bitten': 3, 'bittet': 3, 'schlagen': 4, 'läute': 4, 'läuten': 4, 'rauschte': 3, 'zuschlagen': 4, 'kratze': 3, 'erkundigen': 3, 'schlürfen': 2, 'turmuhr': 4, 'hallend': 4, 'ton': 3, 'antwort': 3, 'erklärung': 3, 'bitte': 3, 'setzen': 3, 'verständigend': 3, 'stiller': 2, 'besprachen': 3, 'inne': 1, 'hielten': 1, 'weine': 3, 'heftig': 4, 'werfen': 3, 'wort': 3,  'zahn': 1, 'erhob': 4, 'erheben': 4, 'fährt': 3, 'fort': 3, 'fahren': 3, 'lamentierte': 3, 'lamentieren': 3, 'frage': 3, 'versprechen': 3, 'versprochen': 3,       }

# Concatenation of the loudness dictionaries
loudness_dict.update(loudness_dict_extension)

#loudness_dict = loudness_dict + loudness_dict_extension # does not work with dictionaries but only with lists, that's why I used the dict.update(new_dict) command

The following function iterates over the data frame column with the lemmatized sound event spans, looking for matching sound words from the key-value pairs in the loudness dictionary. 
A new column is defined in which the found sound words are saved.

In [58]:
# Function to find words in a text that are keys in the sound dictionary
def find_sound_words(text):
    sound_words = []
    for word in text.split():
        if word in loudness_dict:
            sound_words.append(word)
    return sound_words

# Add a column with the list of found sound words for each lemmatized sound_span
final_df['found_sound_words'] = final_df['lemmatized_sound_span'].apply(find_sound_words)

In [59]:
#print(final_df)

In [60]:
final_df[:10]

,filename,sound_span,annotation_class,lemmatized_sound_span,found_sound_words
0,Ring_Max_Vom_alten_Heim.xml,,ambient_sound,,[]
1,Ring_Max_Vom_alten_Heim.xml,Während er sprach,character_sound,während er sprechen,[sprechen]
2,Ring_Max_Vom_alten_Heim.xml,mit denen er sich unterhielt,character_sound,mit der er sich unterhalten,[unterhalten]
3,Ring_Max_Vom_alten_Heim.xml,Er sprach mit ihnen wie ein älterer Freund mit...,character_sound,er sprechen mit ihnen wie ein alt freund mit s...,[sprechen]
4,Ring_Max_Vom_alten_Heim.xml,Beide Aerzte sprachen zuerst von dem vorliegen...,character_sound,beide aerzter sprechen zuerst von der vorliege...,[sprechen]
5,Ring_Max_Vom_alten_Heim.xml,fragte er ihn in dem ihm eigenen salbungsvolle...,character_sound,fragen er ihn in der ihm eigen salbungsvoll tone,"[fragen, tone]"
6,Ring_Max_Vom_alten_Heim.xml,dieser Frage,character_sound,dieser frage,[frage]
7,Ring_Max_Vom_alten_Heim.xml,antwortete er schmunzelnd,character_sound,antworten er schmunzelnd,[antworten]
8,Ring_Max_Vom_alten_Heim.xml,Dabei klopfte der lose Spötter gutmüthig seine...,character_sound,dabei klopfen der los spötter gutmüthig sein c...,[klopfen]
9,Ring_Max_Vom_alten_Heim.xml,und verschrieben mehr zur Beruhigung der Angeh...,character_sound,und verschreiben mehr zu beruhigung der angehö...,[]


The following code counts the empty cells in the column "found_sound_words" to verify how many sound events will not receive a loudness level label because they do not provide a sound word that is part of the loudness level dictionary.

In [62]:
# Count empty and non-empty lists in the 'found_sound_words' column
empty_list_count = final_df['found_sound_words'].apply(lambda x: len(x) == 0).sum()
non_empty_list_count = final_df['found_sound_words'].apply(lambda x: len(x) > 0).sum()

print("Number of empty lists in 'found_sound_words' column:", empty_list_count)
print("Number of non-empty lists in 'found_sound_words' column:", non_empty_list_count)


Number of empty lists in 'found_sound_words' column: 4872
Number of non-empty lists in 'found_sound_words' column: 18121


With the counter, one can verify how many cells remain empty, meaning that the sound event spand did not match with any loudness level dictionary key. 
On the one hand it could be the case that an automatically annotated sound event span maybe actually wasn't a sound event, 
on the other hand, it could be that the sound word from the sound event is not provided in the loudness level dictionary, because it is an unusual sound word or because the lemmatization did not work on a uncommon spelling.
A last reasong could be that the sound event does not provide a sound word because it is a sound metaphor or only an indicated perception of an indirectly indicated sound event.

The following code matches the found sound words with their loudness levels providing lists of values in a new column called "listed_loudness_values". Like this every matched sound word's loudness level gets listed to be taken into account for an average calculation.

In [63]:
# Define a function to map sound words to their loudness levels
def map_to_loudness(sound_words):
    return [loudness_dict[word] for word in sound_words if word in loudness_dict]

# Apply the function to the 'found_sound_words' column and create the new column 'listed_loudness_values'
final_df['listed_loudness_values'] = final_df['found_sound_words'].apply(map_to_loudness)

# Print the DataFrame with the new column
print(final_df)

                                                filename  \
0                            Ring_Max_Vom_alten_Heim.xml   
1                            Ring_Max_Vom_alten_Heim.xml   
2                            Ring_Max_Vom_alten_Heim.xml   
3                            Ring_Max_Vom_alten_Heim.xml   
4                            Ring_Max_Vom_alten_Heim.xml   
...                                                  ...   
22988  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22989  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22990  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22991  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22992  von_Sternberg_Alexander_Ein_literarischer_Aben...   

                                              sound_span annotation_class  \
0                                                           ambient_sound   
1                                      Während er sprach  character_sound   
2                           mit denen er sich un

The following code adds a new column called "average_loudness_value" to the data frame. The function calculate_average_loudness calculates the average of the listed loudness values for each row, ignoring any empty cells. 


In [64]:
# This code calculates a precise value. The next cell concentrates on providing only one digit after the comma using the round-function.

print("This cell can be skipped in favor for the next one.")

skip # comment this word out if you want to execute the cell

# Define a function to calculate the average loudness value
def calculate_average_loudness(listed_loudness_values):
    if not listed_loudness_values:
        return None
    numeric_values = [value for value in listed_loudness_values if pd.notna(value)]
    if not numeric_values:
        return None
    return sum(numeric_values) / len(numeric_values)

# Apply the function to the 'listed_loudness_values' column and create the new column 'average_loudness_value'
final_df['average_loudness_value'] = final_df['listed_loudness_values'].apply(calculate_average_loudness)

# Print the DataFrame with the new column
print(final_df)


This cell can be skipped in favor for the next one.


NameError: name 'skip' is not defined

In [65]:
# Define a function to calculate the average loudness value
def calculate_average_loudness(listed_loudness_values):
    if not listed_loudness_values:
        return None
    numeric_values = [value for value in listed_loudness_values if pd.notna(value)]
    if not numeric_values:
        return None
    # Round the average loudness value to one decimal place
    return round(sum(numeric_values) / len(numeric_values), 1)

# Apply the function to the 'listed_loudness_values' column and create the new column 'average_loudness_value'
final_df['average_loudness_value'] = final_df['listed_loudness_values'].apply(calculate_average_loudness)

# Print the DataFrame with the new column
print(final_df)


                                                filename  \
0                            Ring_Max_Vom_alten_Heim.xml   
1                            Ring_Max_Vom_alten_Heim.xml   
2                            Ring_Max_Vom_alten_Heim.xml   
3                            Ring_Max_Vom_alten_Heim.xml   
4                            Ring_Max_Vom_alten_Heim.xml   
...                                                  ...   
22988  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22989  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22990  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22991  von_Sternberg_Alexander_Ein_literarischer_Aben...   
22992  von_Sternberg_Alexander_Ein_literarischer_Aben...   

                                              sound_span annotation_class  \
0                                                           ambient_sound   
1                                      Während er sprach  character_sound   
2                           mit denen er sich un

## Preparation of the matched sound events with their average loudness levels 

The following code prepares the matched sound events with their average loudness levels as a dictionary for the enrichment the XML elements of the revised files from the source folder with loudness level attributes.

In [67]:
# Filter out rows where "average_loudness_value" is NaN
filtered_df = final_df.dropna(subset=['average_loudness_value'])

# Extracting "sound_span" and "average_loudness_value" columns as a dictionary
sound_loudness_dict = filtered_df.set_index('sound_span')['average_loudness_value'].to_dict()

# Print the dictionary
print(sound_loudness_dict)



{'Während er sprach': 3.0, 'mit denen er sich unterhielt': 3.0, 'Er sprach mit ihnen wie ein älterer Freund mit seinen jüngeren Collegen': 3.0, 'Beide Aerzte sprachen zuerst von dem vorliegenden Falle': 3.0, 'fragte er ihn in dem ihm eigenen salbungsvollen Tone': 3.0, 'dieser Frage': 3.0, 'antwortete er schmunzelnd': 3.0, 'Dabei klopfte der lose Spötter gutmüthig seinem Collegen auf die Schulter': 4.0, 'ohne sich die geringste Wirkung zu versprechen': 3.0, 'so versprach Heim': 3.0, 'fügte er mit ernstem Gesichte hinzu': 3.0, 'Der furchtsame Patient gelobte den strengsten Gehorsam': 3.0, 'Heim erklärte': 3.0, 'und überhäufte seinen Collegen mit Vorwürfen': 4.0, 'rief er erschrocken aus': 4.0, 'antwortete der bescheidene Heim mit einer fast kindlichen Einfalt': 3.0, 'schrie die Frau': 4.0, 'Heim erkundigte sich theilnehmend nicht nur nach dem Befinden': 3.0, 'fügte er im populären Tone hinzu': 3.0, 'Das Weib dankte mit heißen Thränen': 3.0, 'Was der Brave versprochen': 3.0, 'Sie hatte ih

To add the defined average loudness value as a loudness attribute to the XML element surrounding the sound event span in the XML file, you'll need to parse the XML file, locate the relevant element, and add the attribute with the calculated average_loudness_value as its value.
The following code will update the XML file with the calculated loudness attribute value for the relevant XML element. Make sure to run this code for each XML file in your corpus folder and replace the loudness value with the calculated average loudness value for each file.
In the following code:
The regex pattern now has two capturing groups: one for the opening tag of the XML element (<(?:ambient|character)_sound>\s*) and one for the content between the opening tag and the closing tag ({re.escape(sound_span)}\s*<).
The replacement string uses the first capturing group (\1) to preserve the opening tag, extends it with the loudness attribute, and uses the second capturing group (\2) to preserve the content of the xml element closed by the < beginning of the closing element.

In [68]:
#The following finally works, even if I have to add another iteration over the corpus to subsequently delete the still present closing > of the xml opening element tag from the first regex group. This was a painful thing.

# Folder path containing XML files
#folder_path = '/Users/sguhr/Desktop/Diss_notebooks/ner_prediction_sicherheitskopie_20240505_15h/20240501_Subcorpus_1848-55_predicted_for_loudness/revised'

folder_path = folder_path

# Call the function to process XML files in the folder with the sound_loudness_dict
process_xml_files(folder_path, sound_loudness_dict)

#import os
#import re

def process_xml_files(xml_folder, sound_loudness_dict):
    # Iterate over each XML file in the folder
    for filename in os.listdir(xml_folder):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(xml_folder, filename)
            # Read the XML file
            with open(xml_file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()

            # Iterate over keys in the sound_loudness_dict
            for sound_span, loudness_value in sound_loudness_dict.items():
                # Define the regex pattern with two capturing groups
                regex_pattern = fr'(<(?:ambient|character)_sound>\s*)({re.escape(sound_span)}\s*<)'
                
                # Define the replacement string with both capturing groups and loudness attribute
                replacement_string = fr'\1 loudness="{loudness_value}">\2'

                # Find and replace the regex match with the extended XML element
                xml_content = re.sub(regex_pattern, replacement_string, xml_content)

            # Write the modified XML content back to the file
            with open(xml_file_path, 'w', encoding='utf-8') as file:
                file.write(xml_content)





The following code is a further iteration over the folder files to replace the '>' of the opening xml elements that remained in the XML element making it invalid, while its replacement with the addition of the loudness level attribute.
 The following is a simple reg ex match. 

The replace_regex_pattern_in_folder function takes two arguments: input_folder (the path to the input folder containing text files) and output_folder (the path to the output folder where modified files will be saved).
Inside the function, the regex patterns regex_pattern1 and regex_pattern2 are defined.
The function iterates over each file in the input folder using os.listdir and checks if the file ends with the .txt extension.
For each text file found, it reads the content, performs the replacement using re.sub, and writes the modified content to a new file in the output folder.

In [69]:
# This code replaces with a simple reg ex the > of the opening xml element that received the attribute extension via the regex group solution above. 
# Hopefully I will find a better and direct way soon.

#import os
#import re

# Input and output folder paths
input_folder_path = folder_path
output_folder_path = folder_path
#input_folder_path = '/Users/sguhr/Desktop/Diss_notebooks/ner_prediction_sicherheitskopie_20240505_15h/20240501_Subcorpus_1848-55_predicted_for_loudness/revised'
#output_folder_path = '/Users/sguhr/Desktop/Diss_notebooks/ner_prediction_sicherheitskopie_20240505_15h/20240501_Subcorpus_1848-55_predicted_for_loudness/revised'

def postprocess_xml_files_after_regex(input_folder, output_folder):
    # Define the regex patterns
    regex_pattern1 = r'sound> loudness='
    regex_pattern2 = r'sound loudness='

    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.xml'):
            input_file_path = os.path.join(input_folder, filename)
            output_file_path = os.path.join(output_folder, filename)

            # Read the input file
            with open(input_file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # Perform the replacement
            modified_text = re.sub(regex_pattern1, regex_pattern2, text)

            # Write the modified text to the output file
            with open(output_file_path, 'w', encoding='utf-8') as f:
                f.write(modified_text)


# Call the function to perform the replacement for each file in the input folder
postprocess_xml_files_after_regex(input_folder_path, output_folder_path)


In [ ]:
print("The automated loudness level labeling is finished.")